# Chat dialog

Guidance supports chat-based models like ChatGPT and GPT-4 using role tags. These are then converted to the appropriate format for the model (either a JSON API format or special tokens).

In [ ]:
import guidance
import re
guidance.llm = guidance.llms.OpenAI("gpt-4")

# Multi-step chat with hidden blocks

In [ ]:
def parse_best(prosandcons, options):
    best = re.search(r'Best=(\d+)', prosandcons)
    if not best:
        best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
    if best:
        best = int(best.group(1))
    else:
        best = 0
    return options[best]

create_plan = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.
{{~! generate potential options ~}}
Can you please generate one option for how to accomplish this?
Please make the option very short, at most one line.
{{~/user}}
{{#assistant~}}
{{gen 'options' n=5 temperature=1.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{~! generate pros and cons and select the best option ~}}
{{#block hidden=True}}
{{#user~}}
I want to {{goal}}.

Can you please comment on the pros and cons of each of the following options, and then pick the best option?
---{{#each options}}
Option {{@index}}: {{this}}{{/each}}
---
Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option.
{{~/user}}
{{#assistant~}}
{{gen 'prosandcons' temperature=0.0 max_tokens=600}}
{{~/assistant}}
{{/block}}
{{#user~}}
I want to {{goal}}.
{{~! Create a plan }}
Here is my plan:
{{parse_best prosandcons options}}
Please elaborate on this plan, and tell me how to best accomplish it.
{{~/user}}
{{#assistant~}}
{{gen 'plan' max_tokens=500}}
{{~/assistant}}''')

out = create_plan(goal='read more books', parse_best=parse_best)
out

In [ ]:
print('\n'.join(['Option %d: %s' % (i, x) for i, x in enumerate(out['options'])]))

In [ ]:
print(out['prosandcons'])

## Asking help from experts

In [ ]:
experts = guidance(
'''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
I want a response to the following question:
{{query}}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
{{~/user}}
{{#assistant~}}
{{gen 'experts' temperature=0 max_tokens=300}}
{{~/assistant}}
{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
{{~/user}}
{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='What is the meaning of life?')

In [ ]:
experts(query='How can I be more productive?')

### Shorter version

Guidance supports chat models like ChatGPT and GPT-4, as well as open chat models like Vicuna through a unified API based on role tags (e.g. `{{#system}}...{{/system}}`). This allows interactive dialog development that combines rich templating and logical control with modern chat models.

In [ ]:
experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}''')
experts(query='How can I be more productive?', caching=False)

## Agents

In [ ]:
role_simulator = guidance('''
{{#system~}}
You are a helpful assistant
{{~/system}}

{{#user~}}
You will answer the user as {{role}} in the following conversation. At every step, I will provide you with the user input, as well as a comment reminding you of your instructions. Never talk about the fact that you are an AI, even if the user asks you. Always answer as {{role}}.
{{#if first_question}}You can also start the conversation.{{/if}}
{{~/user}}

{{~! The assistant either starts the conversation or not, depending on if this is the first or second agent }}
{{#assistant~}}
Ok, I will follow these instructions.
{{#if first_question}}Let me start the conversation now:
{{role}}: {{first_question}}{{/if}}
{{~/assistant}}

{{~! Then the conversation unrolls }}
{{~#geneach 'conversation'}}
{{#user~}}
User: {{set 'this.input' (await 'input')}}
Comment: Remember, answer as a {{role}}. Start your utterance with {{role}}:
{{~/user}}

{{#assistant~}}
{{gen 'this.response' temperature=0 max_tokens=300}}
{{~/assistant}}
{{~/geneach}}''')

republican = role_simulator(role='Republican')
democrat = role_simulator(role='Democrat')

first_question = '''What do you think is the best way to stop inflation?'''
republican = republican(input=first_question, first_question=None)
democrat = democrat(input=republican["conversation"][-2]["response"].strip('Republican: '), first_question=first_question)
for i in range(2):
    republican = republican(input=democrat["conversation"][-2]["response"].replace('Democrat: ', ''))
    democrat = democrat(input=republican["conversation"][-2]["response"].replace('Republican: ', ''))

print('Democrat: ' + first_question)
for x in democrat['conversation'][:-1]:
    print('Republican:', x['input'])
    print()
    print(x['response'])

## Using a search API

Helper functions for search:

In [ ]:
import os
import diskcache
import pathlib
import requests
import html
from urllib.parse import urlparse
import urllib.parse
import io
import html
import html.parser

curr_dir = './'# pathlib.Path(__file__).parent.resolve()
_bing_cache = diskcache.Cache(f"{curr_dir}/../bing.diskcache")

with open(os.path.expanduser('~/.bing_api_key'), 'r') as file:
    subscription_key = file.read().replace('\n', '')

class MLStripper(html.parser.HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.text = io.StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def bing_search(search_terms, count=10):
    if type(search_terms) == str:
        search_terms = [search_terms]
    search_url = "https://api.bing.microsoft.com/v7.0/search"

    headers = {"Ocp-Apim-Subscription-Key": subscription_key}
    search_results = []
    for search_term in search_terms:
        params = {"q": search_term, "textDecorations": True, "textFormat": "HTML", "cout": count}
        params_key = search_term + "-___-" + str(count)
        if params_key not in _bing_cache or "webPages" not in _bing_cache[params_key]:
            response = requests.get(search_url, headers=headers, params=params)
            response.raise_for_status()
            _bing_cache[params_key] = response.json()
        data = _bing_cache[params_key]["webPages"]["value"]
        for r in data:
            r["snippet_text"] = strip_tags(r["snippet"])
        search_results.extend(data)
    return search_results
def top_snippets(query, n=3):
    results = bing_search(query, count=n)[:n]
    return [{'title': x['name'], 'snippet': x['snippet_text']} for x in results]


In [ ]:
top_snippets("OpenAI founders")

Let's set up a practice round to serve as a one-shot example.

In [ ]:
demo_results = [{'title': 'OpenAI - Wikipedia',
  'snippet': 'OpenAI systems run on the fifth most powerful supercomputer in the world. [5] [6] [7] The organization was founded in San Francisco in 2015 by Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others, [8] [1] [9] who collectively pledged US$ 1 billion. Musk resigned from the board in 2018 but remained a donor.'},
 {'title': 'About - OpenAI',
  'snippet': 'About OpenAI is an AI research and deployment company. Our mission is to ensure that artificial general intelligence benefits all of humanity. Our vision for the future of AGI Our mission is to ensure that artificial general intelligence—AI systems that are generally smarter than humans—benefits all of humanity. Read our plan for AGI'},
 {'title': 'Sam Altman - Wikipedia',
  'snippet': 'Samuel H. Altman ( / ˈɔːltmən / AWLT-mən; born April 22, 1985) is an American entrepreneur, investor, and programmer. [2] He is the CEO of OpenAI and the former president of Y Combinator. [3] [4] Altman is also the co-founder of Loopt (founded in 2005) and Worldcoin (founded in 2020). Early life and education [ edit]'}]
practice_round = guidance(
'''{{#user~}}
Who are the founders of OpenAI?
{{~/user}}
{{#assistant~}}
<search>OpenAI founders</search>
{{~/assistant}}
{{#user~}}
Search results:
{{~#each results}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
The founders of OpenAI are Sam Altman, Reid Hoffman, Jessica Livingston, Elon Musk, Ilya Sutskever, Peter Thiel and others.
{{~/assistant}}''')
practice_round = practice_round(results=demo_results)
practice_round

Now let's write the program:

In [ ]:
def is_search(completion):
    return '<search>' in completion
def search(query):
    return top_snippets(query, n=3)

prompt = guidance('''{{#system~}}
You are a helpful assistant.
{{~/system}}
{{#user~}}
From now on, whenever your response depends on any factual information, please search the web by using the function <search>query</search> before responding. I will then paste web results in, and you can respond.
{{~/user}}
{{#assistant~}}
Ok, I will do that. Let's do a practice round
{{~/assistant}}
{{>practice_round}}
{{#user~}}
That was great, now let's do another one.
{{~/user}}
{{#assistant~}}
Ok, I'm ready.
{{~/assistant}}
{{#user~}}
{{user_query}}
{{~/user}}
{{#assistant~}}
{{gen "query" stop="</search>"}}{{#if (is_search query)}}</search>{{/if}}
{{~/assistant}}
{{#if (is_search query)}}
{{#user~}}
Search results: {{#each (search query)}}
<result>
{{this.title}}
{{this.snippet}}
</result>{{/each}}
{{~/user}}
{{#assistant~}}
{{gen "answer"}}
{{~/assistant}}
{{/if}}''')

prompt = prompt(practice_round=practice_round, search=search, is_search=is_search)

In [ ]:
query = "What is Microsoft's stock price right now?"
p1 = prompt(user_query=query)
p1

In [ ]:
query = "Who is Marco Tulio Ribeiro?"
p2 = prompt(user_query=query)
p2



Let's try a query where it should not search the web

In [ ]:
query = "What is 1+1?"
p3 = prompt(user_query=query)
p3


<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>